In [1]:
import pandas as pd

df = pd.read_csv("C:/Users/kotla/Desktop/Titanic_Survival/tested.csv")  
df.head(10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,0,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,1,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,0,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,1,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,0,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [5]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Fare'] = df['Fare'].fillna(df['Fare'].median())
df.drop(columns=['Cabin'], inplace=True)

In [7]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S


In [9]:
import numpy as np
from scipy import stats

# Select only numerical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Compute Z-scores
z_scores = np.abs(stats.zscore(df[numerical_cols]))

# Identify outliers (Z-score > 3)
outliers = (z_scores > 3).any(axis=1)

print("Number of outliers:", outliers.sum())
print(df[outliers])

Number of outliers: 31
     PassengerId  Survived  Pclass  \
24           916         1       1   
53           945         1       1   
55           947         0       3   
59           951         1       1   
64           956         0       1   
69           961         1       1   
74           966         1       1   
75           967         0       1   
81           973         0       1   
96           988         1       1   
114         1006         1       1   
132         1024         1       3   
139         1031         0       3   
140         1032         1       3   
142         1034         0       1   
154         1046         0       3   
156         1048         1       1   
174         1066         0       3   
184         1076         1       1   
188         1080         1       3   
202         1094         0       1   
214         1106         1       3   
218         1110         1       1   
324         1216         1       1   
342         1234         0 

In [11]:
lower_percentile = 1
upper_percentile = 99

# List of numerical columns to apply capping & flooring
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']

# Applying capping & flooring for each column
for col in num_cols:
    lower_bound = np.percentile(df[col], lower_percentile)
    upper_bound = np.percentile(df[col], upper_percentile)
    
    # Replace values below lower bound and above upper bound
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])

# Check the new distribution
print(df[num_cols].describe())

              Age       SibSp       Parch        Fare
count  418.000000  418.000000  418.000000  418.000000
mean    29.565354    0.425837    0.361244   35.014303
std     12.576615    0.755794    0.778075   51.897518
min      0.933600    0.000000    0.000000    6.447411
25%     23.000000    0.000000    0.000000    7.895800
50%     27.000000    0.000000    0.000000   14.454200
75%     35.750000    1.000000    0.000000   31.471875
max     63.830000    4.000000    4.000000  262.375000


In [13]:
df.drop(columns=['Name'], inplace=True)          #as it is categorical and is not related to project droped this

In [15]:
df.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,0,3,male,34.5,0.0,0.0,330911,7.8292,Q
1,893,1,3,female,47.0,1.0,0.0,363272,7.0000,S
2,894,0,2,male,62.0,0.0,0.0,240276,9.6875,Q
3,895,0,3,male,27.0,0.0,0.0,315154,8.6625,S
4,896,1,3,female,22.0,1.0,1.0,3101298,12.2875,S


In [17]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])  # Convert 'male' and 'female' to 0 and 1
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)  # One-hot encoding


In [19]:
df.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_Q,Embarked_S
0,892,0,3,1,34.5,0.0,0.0,330911,7.8292,True,False
1,893,1,3,0,47.0,1.0,0.0,363272,7.0000,False,True
2,894,0,2,1,62.0,0.0,0.0,240276,9.6875,True,False
3,895,0,3,1,27.0,0.0,0.0,315154,8.6625,False,True
4,896,1,3,0,22.0,1.0,1.0,3101298,12.2875,False,True


In [21]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S']
X = df[features]
y = df['Survived']

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Use .loc to explicitly modify the DataFrame
X.loc[:, ['Age', 'Fare']] = scaler.fit_transform(X[['Age', 'Fare']])


In [33]:
print(X_train.corrwith(y_train))

Pclass       -0.146971
Sex          -1.000000
Age           0.060017
SibSp         0.102794
Parch         0.161071
Fare          0.198173
Embarked_Q    0.129696
Embarked_S   -0.125392
dtype: float64


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = RandomForestClassifier(n_estimators=100,max_depth=5, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')
print(f'Precision: {precision_score(y_test, y_pred):.4f}')

Accuracy: 1.0000
Precision: 1.0000


In [35]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

from sklearn.metrics import accuracy_score
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))


Logistic Regression Accuracy: 1.0


In [37]:
print(X_train.shape, X_test.shape)


(334, 8) (84, 8)


In [39]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

importances = model.feature_importances_
feature_names = X_train.columns

for name, importance in zip(feature_names, importances):
    print(f"{name}: {importance:.4f}")


Pclass: 0.0115
Sex: 0.8468
Age: 0.0367
SibSp: 0.0133
Parch: 0.0168
Fare: 0.0557
Embarked_Q: 0.0137
Embarked_S: 0.0054


In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score

# Drop 'Sex' from training and testing data
X_train = X_train.drop(columns=['Sex'])
X_test = X_test.drop(columns=['Sex'])

# Train a RandomForestClassifier with limited depth
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')
print(f'Precision: {precision_score(y_test, y_pred):.4f}')


Accuracy: 0.6190
Precision: 0.5833


In [61]:
from xgboost import XGBClassifier

# Train an XGBoost model
xgb = XGBClassifier(n_estimators=100, max_depth=5, random_state=42)
xgb.fit(X_train_rfe, y_train)

# Make predictions
y_pred_xgb = xgb.predict(X_test_rfe)

# Evaluate XGBoost
print(f'XGBoost Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}')
print(f'XGBoost Precision: {precision_score(y_test, y_pred_xgb):.4f}')


XGBoost Accuracy: 0.6190
XGBoost Precision: 0.5455
